## Tree GP

In [31]:
import pickle
import yaml
from experiments import gp  as gp_script

if not "conf_gp" in globals():#test if every thing is loaded to not load it twice
    with open("results/log-BipedalWalker-v3-conf_sqdgp-BipedalWalker--2023-03-01-15_43_08/conf.yml") as f:#load the conf associated to the experiments (for env input, output and pset initialization)
        conf_gp = yaml.load(f, Loader=yaml.SafeLoader)

    gp_script.Factory(conf_gp["params"]).init_global_var()

with open("results/log-BipedalWalker-v3-conf_sqdgp-BipedalWalker--2023-03-01-15_43_08\hof-final.pkl", "rb") as input_file:
    hof = pickle.load(input_file)
print(len(hof))
best = hof[0] 

for k, tree in enumerate(best):
    print("OUTPUT:", str(k+1), tree)
print(best.fitness.values, len(best.fitness.rewards))

AttributeError: Can't get attribute 'Individual' on <module 'deap.creator' from 'c:\\python39\\lib\\site-packages\\deap\\creator.py'>

### See agent Behavior

In [23]:
!pip install deap


@@@@@@@@@@@ shell: True


ERROR: Could not find a version that satisfies the requirement 1.3.1 (from versions: none)
ERROR: No matching distribution found for 1.3.1


In [6]:
from GPRL.utils import gp_utils
from GPRL.genetic_programming import team
gp_script.ENV = gp_script.gym.make(conf_gp["params"]["env"])
s = 0
if gp_script.ENV.action_space.shape:

    agent = gp_script.toolbox.compile(best)
else:
    func = gp_script.toolbox.compile(best)
    agent = lambda * s : int(func(*s)[0])
print(best.fitness.values, len(best.fitness.rewards))
print("Agent",agent)
steps = 0
gp_script.ENV.reset()
gp_script.ENV.render()
state = gp_script.ENV.reset()
for k in range(2000):
    state, reward, done, _ = gp_script.ENV.step(agent(*state))
    gp_script.ENV.render()
    s+= reward
    steps += 1
    if done:
        break

print("End! cumulative rewards:", s, " Done?", done, " nb_steps:", k)
gp_script.ENV.close()

(319.791614310743,) 0
Agent <function team_compile.<locals>.func at 0x000001599A386700>
End! cumulative rewards: 317.6686715267835  Done? True  nb_steps: 1275


### Tree GP Graph

In [16]:
import numpy as np
from deap import gp
import pygraphviz as pgv

offset = 0
n = []
e = []
l = {}
for tree in best:#multi-output support
    expr = tree
    nodes, edges, labels = gp.graph(expr)
    n += list(np.array(nodes)+offset)
    e += map(tuple, list(np.array(edges)+offset))
    for key in list(labels.keys()):
        l[key+offset] = labels[key]
    offset += np.max(nodes)+1
nodes = n
edges = e
labels = l

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("img/Tree-GP3.png", prog="dot")

@@@@@@@@@@@ shell: False
@@@@@@@@@@@ shell: False


In [4]:
#!pip install --global-option=build_ext --global-option="-IC:\Program Files\Graphviz\include" --global-option="-LC:\Program Files\Graphviz\lib" pygraphviz


![title](img/Tree-GP2.png)

## Linear GP

In [3]:
#[lambda:print("testing") for _ in range(3)]#[0]

def init_team(size, unit_init):
    team = [unit_init(2) for _ in range(size)]
    return team

init_team(4,lambda x:x+x)

[4, 4, 4, 4]

In [6]:
import pickle
import yaml
from experiments import linGP as linGP_script

if not "conf_lingp" in globals():#test if every thing is loaded to not load it twice
    with open("results/log-MountainCarContinuous-v0-conf_lingp-1631701228.1234944\conf.yml") as f:#load the conf associated to the experiments (for env input, output and pset initialization)
        conf_lingp = yaml.load(f, Loader=yaml.SafeLoader)

    linGP_script.Factory(conf_lingp["params"]).init_global_var()# gives Warning from creator of gp_script

with open("results\log-MountainCarContinuous-v0-conf_lingp-1631701228.1234944\hof-final.pkl", "rb") as input_file:
    hof = pickle.load(input_file)
print(len(hof))
best = hof[-1]
print(best.to_effective(list(range(linGP_script.OUTPUT)))[0], best.fitness.values, len(best.fitness.rewards))

10
   op dst inpt1 inpt2
0 + 0 1 14
1 * 0 5 0
 (98.78012634838458, 2.0) 0


c:\python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### See agent behavior

In [5]:
linGP_script.ENV = linGP_script.gym.make(conf_lingp["params"]["env"])

### Define agent according to the environment
eff, _, _ = best.to_effective(list(range(linGP_script.OUTPUT)))
if linGP_script.ENV.action_space.shape:
    def agent(inputs):
        register = eff.init_register()
        return eff.execute(eff, inputs, register, list(range(linGP_script.OUTPUT)))
else:
    if linGP_script.OUTPUT==1:
        def agent(inputs):
            register = eff.init_register()
            return int(eff.execute(eff, inputs, register, list(range(linGP_script.OUTPUT)))>0)
    else:
        def agent(inputs):
            register = eff.init_register()
            return np.argmax(eff.execute(eff, inputs, register, list(range(linGP_script.OUTPUT))))
s = 0
steps = 0
linGP_script.ENV.reset()
linGP_script.ENV.render()

state = linGP_script.ENV.reset()
for k in range(2000):
    state, reward, done, _ = linGP_script.ENV.step(agent(state))
    linGP_script.ENV.render()
    s+= reward
    steps += 1
    if done:
        break
print("End! cumulative rewards:", s, " Done?", done, " nb_steps:", k)
linGP_script.ENV.close()

AttributeError: 'tuple' object has no attribute 'ndim'

### Linear GP plot as graph

In [6]:
import pygraphviz as pgv

In [8]:
from GPRL.genetic_programming.linearGP import graph
import pygraphviz as pgv

####Give a default name to terminals
register = best.init_register()
terminals_name = [str(round(register[k],2)) for k in range(len(register))]
args = ["ARG"+str(k) for k in range(linGP_script.INPUT)]
terminals_name[best.regCalcSize:best.regCalcSize+best.regInputSize] = args

### Graph attributs
nodes, edges, labels, branch_edges = graph(best, list(range(linGP_script.OUTPUT)), debug=False, terminals_name=terminals_name)

g = pgv.AGraph(directed=True)
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.add_edges_from(branch_edges, style="dashed")
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("img/Linear-GP1.png", prog="dot")

![title](img/Linear-GP1.png)